In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
data_root = r'.\data\train'
print(data_root)

IMAGE_SHAPE = (299, 299)
batch_size = 8

image_generator = ImageDataGenerator(rescale=1/255)
image_data = image_generator.flow_from_directory(
                                    str(data_root), 
                                    target_size=IMAGE_SHAPE,
                                    batch_size=batch_size)


for image_batch, label_batch in image_data:
    print("Image batch shape: ", image_batch.shape)
    print("Label batch shape: ", label_batch.shape)
    break

.\data\train
Found 93 images belonging to 5 classes.
Image batch shape:  (8, 299, 299, 3)
Label batch shape:  (8, 5)


In [9]:
import pandas as pd
import numpy as np
import cv2

In [10]:
tr_data = pd.read_csv(r"./face_file.csv")

In [11]:
tr_data

,images,labels
0,C:\Users\MyLaptop\Documents\Jupyter notebook\R...,0
1,C:\Users\MyLaptop\Documents\Jupyter notebook\R...,0
2,C:\Users\MyLaptop\Documents\Jupyter notebook\R...,0
3,C:\Users\MyLaptop\Documents\Jupyter notebook\R...,0
4,C:\Users\MyLaptop\Documents\Jupyter notebook\R...,0
...,...,...
88,C:\Users\MyLaptop\Documents\Jupyter notebook\R...,4
89,C:\Users\MyLaptop\Documents\Jupyter notebook\R...,4
90,C:\Users\MyLaptop\Documents\Jupyter notebook\R...,4
91,C:\Users\MyLaptop\Documents\Jupyter notebook\R...,4


In [12]:
tr_data = pd.read_csv(r"./face_file.csv").values

In [13]:
train_X = []

for file in tr_data[:,0]:
    img = cv2.imread(file)
    img = cv2.resize(img, dsize=(299, 299))
    img = img / 255.0
    train_X.append(img)

train_X = np.array(train_X)
print(train_X.shape)

(93, 299, 299, 3)


In [14]:
train_y = tr_data[:,1]

In [15]:
train_y.shape

(93,)

In [10]:
type(train_y)

numpy.ndarray

In [16]:
train_X = train_X.astype(np.float64)
train_y = train_y.astype(np.int64)

In [18]:
print(train_X.shape, train_y.shape)

(93, 299, 299, 3) (93,)


In [7]:
import numpy as np

train_X = np.load('./train_X.npy')
train_y = np.load('./train_y.npy')

In [19]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Lambda, Dropout
from tensorflow.keras.optimizers import Adam

# model load
IMAGE_SHAPE = (299,299)

hub_url = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4"


hub_layer = hub.KerasLayer(hub_url,
                           input_shape=IMAGE_SHAPE+(3,))

hub_layer.trainable = False

model = Sequential()
model.add(hub_layer)
model.add(Dense(128, activation=None))
model.add(Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))

model.summary()

adam = Adam(lr=1e-3, decay=1e-6)
model.compile(optimizer=adam, loss=tfa.losses.TripletSemiHardLoss())

# steps_per_epoch = np.ceil(image_data.samples/image_data.batch_size)
history = model.fit(train_X, train_y,
                    epochs=10)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_3 (KerasLayer)   (None, 2048)              21802784  
_________________________________________________________________
dense_2 (Dense)              (None, 128)               262272    
_________________________________________________________________
lambda_2 (Lambda)            (None, 128)               0         
Total params: 22,065,056
Trainable params: 262,272
Non-trainable params: 21,802,784
_________________________________________________________________
Epoch 1/10
3/3 [==============================] - 4s 1s/step - loss: 0.9687
Epoch 2/10
3/3 [==============================] - 0s 116ms/step - loss: 0.9037
Epoch 3/10
3/3 [==============================] - 0s 116ms/step - loss: 0.8403
Epoch 4/10
3/3 [==============================] - 0s 116ms/step - loss: 0.7550
Epoch 5/10
3/3 [==============================] - 0s 